<a href="https://colab.research.google.com/github/kirankumarkv/Food_Image_Classifier/blob/main/Food_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import random
import numpy as np
from pathlib import Path
import os.path

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
!cp /content/model_weights.h5 /content/drive/MyDrive/.


In [18]:
from pathlib import Path

image_dir = Path('/content/drive/MyDrive/FoodImage/')

image_df = []

# Iterate over the subdirectories
for subdirectory in image_dir.iterdir():
    if subdirectory.is_dir():
        label = subdirectory.name  # Use the subdirectory name as the label
        # Iterate over the images in the subdirectory
        for image_path in subdirectory.glob('*.jpg'):  # Modify the pattern according to your image file extension
            filepath = str(image_path)  # Convert the Path object to a string
            image_df.append((filepath, label))  # Append the (filepath, label) tuple to the image_df list


In [ ]:
image_df

In [32]:
labels = [label for _, label in image_df]

# Count the occurrences of each label
label_counts = {}
for label in labels:
    if label not in label_counts:
        label_counts[label] = 0
    label_counts[label] += 1

# Print the label counts
for label, count in label_counts.items():
    print(f"{label}: {count}")

Bread: 90
ApplePie: 93
FriedRice: 88
BagelSandwich: 45
Bibimbop: 92
Pork: 89


In [33]:
# Split the image_df into train, validation, and test DataFrames
train_df, temp_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)
val_df, test_df = train_test_split(temp_df, train_size=0.5, shuffle=True, random_state=1)

# Convert train_df, val_df, and test_df to Pandas DataFrames
train_df = pd.DataFrame(train_df, columns=['Filepath', 'Label'])
val_df = pd.DataFrame(val_df, columns=['Filepath', 'Label'])
test_df = pd.DataFrame(test_df, columns=['Filepath', 'Label'])

# Set up the image data generator with data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    rotation_range=40,         # Increased rotation range
    width_shift_range=0.3,     # Increased width shift range
    height_shift_range=0.3,    # Increased height shift range
    shear_range=0.2,           # Increased shear range
    zoom_range=0.2,            # Increased zoom range
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Flow from dataframe for train data
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='Filepath',  # Column name containing the filepaths
    y_col='Label',  # Column name containing the labels
    target_size=(224, 224),  # Adjust the target size as needed
    batch_size=32,  # Adjust the batch size as needed
    class_mode='categorical',  # Set the class_mode based on your problem
    subset='training',  # Specify training subset
    shuffle=True,  # Shuffle the data during training
    seed=1  # Set a seed for reproducibility
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
)

# Flow from dataframe for validation data
val_generator = val_datagen.flow_from_dataframe(
    test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Flow from dataframe for test data
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Do not shuffle the data during testing
)


Found 347 validated image filenames belonging to 6 classes.
Found 75 validated image filenames belonging to 6 classes.
Found 75 validated image filenames belonging to 6 classes.


In [36]:
from tensorflow.keras.applications import InceptionV3

# Create a InceptionV3 model

model = models.Sequential()

# Add the pretrained InceptionV3 model as the base
pretrained_model = InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
model.add(pretrained_model)

# Add more layers to the model
model.add(layers.GlobalAveragePooling2D())  # Add GlobalAveragePooling2D to reduce spatial dimensions
model.add(layers.Dense(1024, activation='relu'))  # Add a dense layer with 1024 units and ReLU activation
model.add(layers.Dropout(0.5))  # Add dropout for regularization
model.add(layers.Dense(512, activation='relu'))  # Add another dense layer with 512 units and ReLU activation
model.add(layers.Dense(256, activation='relu'))  # Add another dense layer with 256 units and ReLU activation
model.add(layers.Dense(6, activation='softmax'))  # Add the output layer with 6 units for your classes and softmax activation

# Print the model summary to see the updated architecture
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_3   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_12 (Dense)            (None, 1024)              2098176   
                                                                 
 dropout_3 (Dropout)         (None, 1024)              0         
                                                                 
 dense_13 (Dense)            (None, 512)               524800    
                                                                 
 dense_14 (Dense)            (None, 256)               131328    
                                                      

In [24]:
# Set up the ModelCheckpoint callback
checkpoint_filepath = '/content/drive/MyDrive/model_weights.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',  # You can choose the metric to monitor for saving weights
    mode='min',          # 'min' for loss, 'max' for accuracy, etc.
    save_best_only=True  # Only save the best weights based on the monitored metric
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Set the number of steps per epoch and validation steps
train_steps_per_epoch = len(train_generator)
val_steps = len(test_generator)

# Train the model using the data generators
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    steps_per_epoch=train_steps_per_epoch,
    validation_steps=val_steps,
    callbacks=[
        model_checkpoint_callback,  # Add the ModelCheckpoint callback

    ]
        #tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3,
         #   restore_best_weights=True),


)


Epoch 1/50
11/11 [==============================] - 82s 3s/step - loss: 1.7076 - accuracy: 0.3314 - val_loss: 3.1800 - val_accuracy: 0.2400
Epoch 2/50
11/11 [==============================] - 33s 3s/step - loss: 1.1134 - accuracy: 0.6340 - val_loss: 2.1335 - val_accuracy: 0.2533
Epoch 3/50
11/11 [==============================] - 31s 3s/step - loss: 0.9621 - accuracy: 0.6830 - val_loss: 16.8913 - val_accuracy: 0.2133
Epoch 4/50
11/11 [==============================] - 31s 3s/step - loss: 0.9481 - accuracy: 0.7147 - val_loss: 3.1931 - val_accuracy: 0.1333
Epoch 5/50
11/11 [==============================] - 32s 3s/step - loss: 0.9633 - accuracy: 0.7233 - val_loss: 2.9841 - val_accuracy: 0.3200
Epoch 6/50
11/11 [==============================] - 32s 3s/step - loss: 0.8016 - accuracy: 0.7435 - val_loss: 10.0311 - val_accuracy: 0.3200
Epoch 7/50
11/11 [==============================] - 32s 3s/step - loss: 0.8216 - accuracy: 0.7550 - val_loss: 4.9432 - val_accuracy: 0.2133
Epoch 8/50
11/11 [

In [29]:
# Evaluate the model on the test data generator
results = model.evaluate(test_generator, verbose=0)

print("Test Loss:", results[0])
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

Test Loss: 1.6646661758422852
Test Accuracy: 58.67%


In [31]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Get predictions using the test data generator
predictions = np.argmax(model.predict(test_generator), axis=1)

# Get true labels from the test data generator
true_labels = test_generator.classes

# Compute confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Compute classification report
class_names = list(test_generator.class_indices.keys())
clr = classification_report(true_labels, predictions, target_names=class_names, zero_division=0)

print("Confusion Matrix:")
print(cm)

print("\nClassification Report:")
print(clr)


3/3 [==============================] - 5s 1s/step
Confusion Matrix:
[[ 4  0  6  3  0  0]
 [ 0  5  2  0  0  0]
 [ 0  0 12  0  0  0]
 [ 0  1  2 13  0  0]
 [ 0  3  2  0  7  0]
 [ 0  3  7  2  0  3]]

Classification Report:
               precision    recall  f1-score   support

     ApplePie       1.00      0.31      0.47        13
BagelSandwich       0.42      0.71      0.53         7
     Bibimbop       0.39      1.00      0.56        12
        Bread       0.72      0.81      0.76        16
    FriedRice       1.00      0.58      0.74        12
         Pork       1.00      0.20      0.33        15

     accuracy                           0.59        75
    macro avg       0.75      0.60      0.56        75
 weighted avg       0.79      0.59      0.57        75

